In [8]:
import streamlit as st
import pandas as pd
import numpy as np
import datetime
import hopsworks
from functions import figure, retrieve
import os
import pickle
import plotly.express as px
import json
from datetime import datetime
import os


# Real data
today = datetime.today().strftime('%Y-%m-%d')
df = retrieve.get_merged_dataframe()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160344
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.28s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.04s) 


In [17]:
import hopsworks
import os

from functions import util
api_key = os.getenv('HOPSWORKS_API_KEY')
project_name = os.getenv('HOPSWORKS_PROJECT')
project = hopsworks.login(project=project_name, api_key_value=api_key)
util.purge_project(project)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160344
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
No air_quality_fv feature view found
No air_quality feature group found
No weather feature group found
No aq_predictions feature group found
No air_quality_xgboost_model model found
Connected. Call `.close()` to terminate connection gracefully.
Deleted secret SENSOR_LOCATION_JSON


In [ ]:
def backfill_predictions_for_monitoring(weather_fg, air_quality_df, monitor_fg, model):
    weather_df = weather_fg.read()
    weather_df = weather_df.sort_values(by=['date'], ascending=True)
    weather_df['date'] = weather_df['date'].dt.tz_convert(None).astype('datetime64[ns]')
    air_quality_df_filter = air_quality_df[['date', 'past_air_quality']]
    monitor_fg_filter = monitor_fg.read()[['date','past_air_quality']]
    combined_df = pd.concat([air_quality_df_filter, monitor_fg_filter])
    combined_df['date'] = pd.to_datetime(combined_df['date'], utc=True)
    combined_df['date'] = combined_df['date'].dt.tz_convert(None).astype('datetime64[ns]')
    features_df = pd.merge(weather_df, combined_df, on='date', how='left')
    
    features_df = features_df.tail(10)
    features_df['predicted_pm25'] = model.predict(features_df[['past_air_quality','temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant']])
    air_quality_df['date'] = pd.to_datetime(air_quality_df['date'])
    # features_df['date'] = features_df['date'].dt.tz_convert(None).astype('datetime64[ns]')
    
    df = pd.merge(features_df, air_quality_df[['date','pm25','street','country']], on="date")
    df['days_before_forecast_day'] = 1
    hindcast_df = df
    df = df.drop('pm25', axis=1)
    monitor_fg.insert(df, write_options={"wait_for_job": True})
    return hindcast_df

9